# Introduction

The *Real or Not? NLP with Disaster Tweets* competitions offers a neat opportunity to see how different approaches to natural language processing work when compared to one another. In this notebook, we'll look at how to start examining NLP data and performing some rudimentary second-order feature engineering. Here's a breakdown of what this notebook covers:

1. Perform an initial exploration of some simple fields.
2. Clean and normalize the data set.
3. Extract first-order features and examine how useful they are.
4. Perform rudimentary natural language processing on the text field.
5. Evaluate our natural language model.
6. Use the model and make predictions that we can submit to the competition.

# 1. Importing the Data

The first step in the process is to import our training data so we can see what kinds of information we have to work with. For this project, we'll start by importing the entire training dataset into a single Pandas dataframe.

In [ ]:
import time
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv("../input/nlp-getting-started/train.csv")
display(train)

test = pd.read_csv("../input/nlp-getting-started/test.csv")
display(test)

# 1.1 Eliminating Duplicates

One thing we should do is check to see if we have duplicated or conflicting data. Here's an easy way to check for textual duplicates against the `target` - which is the class we're trying to predict.

In [ ]:
duplicates = pd.concat(x for _, x in train.groupby(["text"]) if len(x) > 1)
with pd.option_context("display.max_rows", None, "max_colwidth", 240):
    display(duplicates[["id", "target", "text"]])

It looks like we have quite a few duplicates. In some instances, the duplicates resolve to the same target class, but in others such as duplicate indexes `5620` and `5641`, we have the same tweet belonging to two different classes. For those instances where the tweet belongs to the same class, we can simply delete the duplicates.

In [ ]:
train.drop(
    [
        6449, 7034, 3589, 3591, 3597, 3600, 3603, 3604, 3610, 3613, 3614, 119, 106, 115,
        2666, 2679, 1356, 7609, 3382, 1335, 2655, 2674, 1343, 4291, 4303, 1345, 48, 3374,
        7600, 164, 5292, 2352, 4308, 4306, 4310, 1332, 1156, 7610, 2441, 2449, 2454, 2477,
        2452, 2456, 3390, 7611, 6656, 1360, 5771, 4351, 5073, 4601, 5665, 7135, 5720, 5723,
        5734, 1623, 7533, 7537, 7026, 4834, 4631, 3461, 6366, 6373, 6377, 6378, 6392, 2828,
        2841, 1725, 3795, 1251, 7607
    ], inplace=True
)
duplicates = pd.concat(x for _, x in train.groupby(["text"]) if len(x) > 1)
with pd.option_context("display.max_rows", None, "max_colwidth", 240):
    display(duplicates[["id", "target", "text"]])

Now we're facing a challenge. We could keep one duplicate with one target class, but we don't have access to the method by which the dataset creators used to mark up real versus not real disaster tweets. They may have had access to more information than us, so we have to be careful if we alter the dataset - we could introduce personal bias. While it may be tempting to try to keep some of the data (e.g. `that horrible sinking feeling when you've been at home on your phone for a while and you realise its been on 3G this whole time` seems like it should be marked as `not real`), the better approach is to simply delete the offending duplicates. While this cuts our training size down, we ensure we haven't inadventently introduced bias to the dataset.

In [ ]:
train.drop(
    [
        4290, 4299, 4312, 4221, 4239, 4244, 2830, 2831, 2832, 2833, 4597, 4605, 4618, 4232, 4235, 3240,
        3243, 3248, 3251, 3261, 3266, 4285, 4305, 4313, 1214, 1365, 6614, 6616, 1197, 1331, 4379, 4381,
        4284, 4286, 4292, 4304, 4309, 4318, 610, 624, 630, 634, 3985, 4013, 4019, 1221, 1349, 6091, 6094, 
        6103, 6123, 5620, 5641
    ], inplace=True
)

# 1.2 Keyword and Location Normalization

It looks like we need to do a little cleanup here. Both `keyword` and `location` fields are meant to be interpreted as strings. While we're at it, we should probably convert them all to lowercase for ease of processing. We'll also fill all missing values (`<NA>` values) in `keyword` and `location` with the empty string. If we look at the `keyword` strings, we find that some entries have `%20` instead of a space. We should also stem the `keyword` field so we can collapse similar keywords into a single keyword (for example, `death` and `deaths` would become `death`). Let's go ahead and make those changes to the dataframe. 

In [ ]:
from gensim.parsing.preprocessing import stem_text

def clean_location_keyword(df):
    df["location"] = df["location"].astype("string").str.lower()
    df["location"].fillna("", inplace=True)
    df["keyword"] = df["keyword"].astype("string").str.lower()
    df["keyword"].replace(regex=r"\%20", value=" ", inplace=True)
    df["keyword"].fillna("", inplace=True)
    df["keyword"] = df["keyword"].apply(stem_text)

clean_location_keyword(train)
clean_location_keyword(test)

# 2. Looking at Class Imbalance

It looks like we have 7,613 training samples. Let's see how many tweets we have that are examples of disaster versus those that are not. What we're looking at is whether or not we have a balance between samples that are both real examples of disasters, and those that are not.

In [ ]:
counts = pd.DataFrame(train["target"].value_counts())
counts.rename(columns={"target": "Samples"}, index={0: "Not Real", 1: "Real"}, inplace=True)
ax = sns.barplot(x=counts.index, y=counts.Samples)
for p in ax.patches:
    height = p.get_height()
    ax.text(
        x=p.get_x()+(p.get_width()/2),
        y=height,
        s=round(height),
        ha="center"
    )

For this particular set of data, it looks like we have a slightly skewed distribution between the two classes. In this instance, we'll have to be careful with any machine learning algorithm we use, since we have more tweets that do not pertain to disasters than we do that contain real disasters. 

# 3. Looking at Keywords

Let's take a closer look at what kind of information we have in the `keyword` field, specifically what unique values we have.

In [ ]:
with pd.option_context("display.max_rows", None):
    display(train["keyword"].unique())

There are a few things we can collapse. For example, `arson` and `arsonist` can be collapsed to `arson`. ALet's go ahead and make a few of these changes.

In [ ]:
def collapse_keywords(x):
    if x == "arsonist":
        return "arson"
    if x == "blaze":
        return "ablaz"
    if x == "bloodi":
        return "blood"
    if x == "build burn" or x == "burn build":
        return "build on fire"
    if x == "blew up":
        return "blown up"
    if x == "colli":
        return "collid"
    if x == "explo":
        return "explod"
    if x == "hailstorm":
        return "hail"
    if x == "injuri":
        return "injur"
    if x == "panick":
        return "panic"
    if x == "suicid bomber":
        return "suicid bomb"
    if x == "wildfir":
        return "wild fire"
    return x

train["keyword"] = train["keyword"].apply(lambda x: collapse_keywords(x))
test["keyword"] = test["keyword"].apply(lambda x: collapse_keywords(x))
with pd.option_context("display.max_rows", None):
    display(train["keyword"].unique())

Now let's look at the keyword in relation to whether their target is real `1` or not real `0`. Let's just take a look at the first 50 rows or so.

In [ ]:
with pd.option_context("display.max_rows", None):
    display(pd.DataFrame(data=train[["id", "keyword", "target"]].groupby(["keyword", "target"]).count()).rename(columns={"id": "count"}).head(50))

We can see that there are certain keywords that are strongly tied to one class. For example, `airplan accid` is very strongly associated with the real disaster target - we see it appear 35 times, and 30 of those times it is a real disaster, while only 5 times it is not. This is good news, since it suggests there are likely keywords here that will provide separation between classes.

Once last thing we will have to do with the keywords is categorize them. Most machine learning algorithms expect categorical data to be in the form of an numeric. We'll convert the keyword into a category type, and save the category code as a new column, and use that for our machine learning technique.

In [ ]:
train["keyword"] = train["keyword"].astype("category")
train["keyword_cat"] = train["keyword"].cat.codes
test["keyword_cat"] = test["keyword"].apply(lambda x: train["keyword"].cat.categories.get_loc(x) if x in train["keyword"].cat.categories.to_list() else 0)
train

# 4. Looking at Location

Let's take look at the first 500 entries in the location field and see what we're working with.

In [ ]:
print([location for location in train["location"]][:500])

In some instances we have countries, others include states, and some include cities. Yet others include junk data such as `global` as well as `Twitter Lockout in progress`. We'll need a way to clean and normalize this data so that it's a little more useful to us. Normalizing this data may turn out to be beneficial, but based on how messy the field is, it may not be worthwhile to spend huge amounts of time trying to clean it. As it stands, let's see if we can use the Python package `pycountry` to help us sort out some of this data. What we'll do is try and sort out real locations from ones that are not real. We can compare what is in the `location` field to subdivision data from `pycountry`. If we get a match, we'll save the state and country to some new columns on the dataframe. If we don't get a match, we'll try and do a little more processing. If we have two floating point numbers, we probably have a set of geo coordinates, so we can mark that as not being location spam. Other than that, we can't do much with the data, so we'll flag it as probable spam.

In [ ]:
import re

from pycountry import subdivisions

def clean_state_country(df):
    subs = [subdivision.name.lower() for subdivision in subdivisions]
    countries = [subdivision.country_code for subdivision in subdivisions]
    country = []
    state = []
    location_spam = []
    for _, row in df.iterrows():
        match_found = False
        is_spam = 0
        if row["location"] == "":
            country.append("")
            state.append("")
        else:
            for index, subdivision in enumerate(subs):
                if subdivision in row["location"]:
                    country.append(countries[index])
                    state.append(subdivision)
                    match_found = True
                    break
            if not match_found:
                country.append("")
                state.append("")
                split_data = row["location"].replace(" ", "").split(",")
                is_spam = 1
                if len(split_data) == 2:
                    if re.match(r"[\-]*[0-9]+\.[0-9]+", split_data[0]) and re.match(r"[\-]*[0-9]+\.[0-9]+", split_data[0]):
                        is_spam = 0
        location_spam.append(is_spam)
    df["country"] = country
    df["state"] = state
    df["location_spam"] = location_spam
    
clean_state_country(train)
clean_state_country(test)

train["country"] = train["country"].astype("category")
train["country_cat"] = train["country"].cat.codes
train["state"] = train["state"].astype("category")
train["state_cat"] = train["state"].cat.codes

# Apply the same category codes to the testing dataset
test["state_cat"] = test["state"].apply(lambda x: train["state"].cat.categories.get_loc(x) if x in train["state"].cat.categories.to_list() else 0)
test["country_cat"] = test["country"].apply(lambda x: train["country"].cat.categories.get_loc(x) if x in train["country"].cat.categories.to_list() else 0)
display(train[(train["location_spam"] == 1)])
display(train[(train["country_cat"] != 0)])

Now that we have country and state information, we should be able to look at those fields the same way we examined keywords. Let's take a look what happens.

In [ ]:
with pd.option_context("display.max_rows", None):
    display(pd.DataFrame(data=train[["id", "country", "target"]].groupby(["country", "target"]).count()).rename(columns={"id": "count"}).head(50))

There are quite a number of entries for which we have no country information. The first row shows us the empty string, which is the number of rows without country information. The total is 5,353 entries, which is more than half of our available training data. For entries with countries, we're seeing somewhat equal splits between real and not real disasters. This is to be expected, as people geotag tweets from all countries whether or not they are actually disasters. It's unlikely that real disasters would exclusively be geotagged. This is probably the same for state information. Let's take a look.

In [ ]:
with pd.option_context("display.max_rows", None):
    display(pd.DataFrame(data=train[["id", "state", "target"]].groupby(["state", "target"]).count()).rename(columns={"id": "count"}).head(50))

As predicted, we're missing the same amount of state information. Looking at the low counts for each state, we're probably not going to get very useful information from this field, but we'll keep it intact for now.

# 5. Simple Feature Engineering

Before we look directly at the text as a feature, let's think about some of the other first-order information we can extract from it. Here are a few features that may be informative:

* Total length of the text
* Average word length
* Number of `@` mentions
* Number of hashtags
* Number of numeric values in the text (excluding timestamps)
* Number of URLs in the text
* Number of timestamps in the text

Let's go ahead and extract these fields.

In [ ]:
import re

def engineer_features(df):
    df["total_length"] = df["text"].apply(len)
    df["avg_word_length"] = df["text"].apply(lambda x: round(sum(len(word) for word in x.split()) / len(x.split())))
    df["num_ats"] = df["text"].apply(lambda x: x.count("@"))
    df["num_hashtags"] = df["text"].apply(lambda x: x.count("#"))
    df["num_numeric"] = df["text"].apply(lambda x: len(re.findall(r"\w[0-9,]+\w", x)))
    df["num_urls"] = df["text"].apply(lambda x: x.count("http"))
    df["num_timestamps"] = df["text"].apply(lambda x: len(re.findall(r"[0-9]+:[0-9]+", x)))

engineer_features(train)
engineer_features(test)
train

# 6. Pre-processing Text

For our textual analysis to be useful, we'll have to perform some pre-processing on the text first to make it easier to work with. Here's what we're going to do:

* Remove words that don't have any value, such as `the`, `of`, `and` (stopword removal)
* We're going to strip out any links since they are `tco` encoded for Twitter
* We'll remove all punctuation
* We'll remove the numerics
* We'll remove multiple whitespaces
* Stem the text

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_multiple_whitespaces, strip_numeric, stem_text

def normalize_text(df):
    normalized_text = []

    for _, row in df.iterrows():
        new_text = row["text"].lower()
        new_text = remove_stopwords(new_text)
        new_text = re.sub(r"http\S+", "", new_text)
        new_text = strip_punctuation(new_text)
        new_text = strip_numeric(new_text)
        new_text = strip_multiple_whitespaces(new_text)
        new_text = stem_text(new_text)
        normalized_text.append(new_text.strip())

    df["normalized_text"] = normalized_text

normalize_text(train)
normalize_text(test)
train

# 7. Training and Validating the Classifier

Now it's time to acutally build a classifier and see how well it does. To evaluate how well our features are working, we're going to split up our training data into a training set and a validation set. We'll do this 5 times for a 5-fold cross validation. Our difference between sets gives us an idea how robust our model actually is to variations in training data. To handle our textual data, we're going to treat it as a bag-of-words and use a regression model. To represent the words, we'll use a TF/IDF vector (term frequency / inverse document frequency). Once we build the regressor, we'll use that to make predictions, and use the predicted outputs, along with our other features in a light GBM model to make actual predictions on the test data. 

In [ ]:
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
skf = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)

features = [
    "keyword_cat", "country_cat", "state_cat", "location_spam", "total_length", "avg_word_length", 
    "num_ats", "num_hashtags", "num_numeric", "num_urls", "num_timestamps"
]

params = {
    "learning_rate": 0.01, 
    "verbose": 100, 
    "random_state": 2020, 
    "metric": "average_precision", 
    "verbose": -1,
    "boosting_type": "dart",
    "num_leaves": 40,
    "cat_column": "0,1,2,3", # array features 0 (keyword_cat), 1 (country_cat), 2 (state_cat), 3 (location_spam) are categorical
    "num_iterations": 200
}

importances = pd.DataFrame()
best_score = 0.0
best_model = None
best_sgd = None

for fold, (train_index, test_index) in enumerate(skf.split(train, train["target"])):
    print("-------> fold {} <--------".format(fold + 1))
    print(": Build regression model")
    x_train, x_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[test_index])
    y_train, y_valid = train["target"].iloc[train_index], train["target"].iloc[test_index]
    
    x_train_tfidf = vectorizer.fit_transform(x_train["normalized_text"]).astype(np.float32)
    x_valid_tfidf = vectorizer.transform(x_valid["normalized_text"]).astype(np.float32)
    
    regressor = LogisticRegression(random_state=2020, max_iter=1000)
    regressor.fit(x_train_tfidf, y_train)
    
    x_train_features = pd.DataFrame(x_train[features])
    x_train_features["sgd_score"] = regressor.predict(x_train_tfidf)
    x_valid_features = pd.DataFrame(x_valid[features])
    x_valid_features["sgd_score"] = regressor.predict(x_valid_tfidf)
    
    print(": Build GBM model")
    model = lgb.LGBMClassifier(
        **params,
    )
    model.fit(
        x_train_features, 
        y_train,
        eval_set=[(x_valid_features, y_valid)],
        verbose=100,
    )

    train_predictions = model.predict(x_valid_features)
    
    imp_df = pd.DataFrame()
    imp_df['feature'] = x_valid_features.columns
    imp_df['gain'] = model.booster_.feature_importance(importance_type='gain')
    imp_df['fold'] = fold + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    print(classification_report(y_valid, train_predictions, target_names=["Not Real", "Real"]))
    score = model.score(x_valid_features, y_valid)
    if score > best_score:
        print("--> This model is the best so far {:0.5}".format(score))
        best_model = model
        best_score = score
        # Best is 0.82565 - Logistic Regression default

# 8. Feature Performance

We can take a look and see how our various features are performing.

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

importances['gain_log'] = np.log1p(importances['gain'])
mean_gain = importances[['gain', 'feature']].groupby('feature').mean()
importances['mean_gain'] = importances['feature'].map(mean_gain['gain'])

plt.figure(figsize=(14, 35))
_ = sns.barplot(x='gain_log', y='feature', data=importances.sort_values('mean_gain', ascending=False))

# 9. Building and Submitting the Final Model

Let's go ahead and build a model that uses all of the data, and takes all the features we've examined so far. Once the model is built, we can submit the result.

In [ ]:
vectorizer = TfidfVectorizer(binary=True)

print(": Build regression model")
train_tfidf = vectorizer.fit_transform(train["normalized_text"]).astype(np.float32)

regressor = LogisticRegression(random_state=2020, max_iter=1000)
regressor.fit(train_tfidf, train["target"])

train_features = pd.DataFrame(train[features])
train_features["sgd_score"] = regressor.predict(train_tfidf)

print(": Build GBM model")
model = lgb.LGBMClassifier(
    **params,
)
model.fit(
    train_features, 
    train["target"],
    verbose=100,
)

Here is the code to run the predictions on the test data, and build the submission file.

In [ ]:
test_tfidf = vectorizer.transform(test["normalized_text"]).astype(np.float32)
test_features = pd.DataFrame(test[features])
test_features["sgd_score"] = regressor.predict(test_tfidf)
predictions = model.predict(test_features)
submission = pd.DataFrame({"id": test["id"], "target": predictions})
submission.to_csv("submission.csv", index=False)